In [1]:
# https://chokkan.github.io/mlnote/classification/01binary.html
# 線形二値分類

In [2]:
# ロジスティック回帰

# シグモイド関数の計算を大きくさせない実装方法
def sigmoid(a):
    if 0 <= a:
        return 1 / (1 + np.exp(-a))
    else:
        return 1. - 1 / (1 + np.exp(a))

In [3]:
# スパムメールの判別問題(DLから展開まで / 以下コメントアウト)
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
# !unzip smsspamcollection.zip
# !head SMSSpamCollection

In [4]:
import collections

def tokenize(s):
    return [t.rstrip('.') for t in s.split(' ')]

def vectorize(tokens):
    return collections.Counter(tokens)

def readiter(fi):
    for line in fi:
        fields = line.strip('\n').split('\t')
        x = vectorize(tokenize(fields[1]))
        y = fields[0]
        yield x, y

with open('SMSSpamCollection') as fi:
    D = [d for d in readiter(fi)]

In [5]:
# 訓練データ(90%)と評価データ(10%)に分割
from sklearn.model_selection import train_test_split

Dtrain, Dtest = train_test_split(D, test_size=0.1, random_state=0)

In [6]:
print(len(Dtrain), len(Dtest))

5016 558


In [7]:
# データ形式の変換
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import DictVectorizer# 特徴をキー、値をバリューとする辞書オブジェクトから特徴ベクトルに変換する

VX = DictVectorizer()
VY = LabelEncoder()

Xtrain = VX.fit_transform([d[0] for d in Dtrain])
Ytrain = VY.fit_transform([d[1] for d in Dtrain])
Xtest = VX.transform([d[0] for d in Dtest])
Ytest = VY.transform([d[1] for d in Dtest])

In [8]:
Dtrain[10]

(Counter({'I': 1,
          'take': 1,
          'it': 2,
          'we': 3,
          "didn't": 1,
          'have': 2,
          'the': 1,
          'phone': 1,
          'callon': 1,
          'Friday': 1,
          'Can': 1,
          'assume': 1,
          "won't": 1,
          'this': 1,
          'year': 1,
          'now?': 1}),
 'ham')

In [9]:
print(Xtrain[10])
print(VX.feature_names_[12653])#疎行列の12653列目の値が3.0なのでその値を入力して単語が何かを調べることができる
print(Ytrain[10])# hamが0 spamが1に分類されていることがわかる

  (0, 1831)	1.0
  (0, 2385)	1.0
  (0, 2769)	1.0
  (0, 5546)	1.0
  (0, 6110)	1.0
  (0, 6923)	1.0
  (0, 8101)	2.0
  (0, 8587)	2.0
  (0, 9821)	1.0
  (0, 10231)	1.0
  (0, 11832)	1.0
  (0, 11957)	1.0
  (0, 12014)	1.0
  (0, 12653)	3.0
  (0, 12862)	1.0
  (0, 13030)	1.0
we
0


In [10]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(loss='log')# デフォルトだと線形分類モデルを確率的勾配降下法で求める。引数にlogを指定するとロジスティック回帰で求めてくれる
model.fit(Xtrain, Ytrain)

SGDClassifier(loss='log')

In [11]:
model.predict(Xtest[0])

array([0])

In [12]:
# ham および spamの予測をする確率を出力
model.predict_proba(Xtest[0])

array([[0.99638588, 0.00361412]])

In [13]:
model.score(Xtest, Ytest)

0.9695340501792115

In [14]:
msg = "Your account has been credited with 500 FREE Text Messages."
model.predict_proba(VX.transform(vectorize(tokenize(msg))))
# 結果スパムに分類された

array([[0.17683535, 0.82316465]])

In [15]:
model.coef_

array([[-0.79413186, -0.13094001, -0.00331027, ...,  0.36239771,
        -0.27695296, -0.00163919]])

In [16]:
# 特徴を表す単語とその重みのタプルからなるリストを作成し、重みが小さい順に並べたものを変数Fに格納する。
F = sorted(zip(VX.feature_names_, model.coef_[0]), key=lambda x: x[1])

In [17]:
# ham が 0 で spam が 1 なので、以下の単語を含むメッセージはhamだと予測されやすくなる（上位20単語）
F[:20]# 重みの値が負に大きいトップ20

[('&lt;#&gt;', -1.291071897653726),
 ('me', -1.1984882792408),
 ('him', -1.1577750969718144),
 ('&amp;', -1.1216312213976345),
 ('I', -1.1035737965370789),
 ('later', -1.0541651117599518),
 ('Happy', -1.0269759672519978),
 ('What', -1.000594325026295),
 ('And', -0.983391730559116),
 ('simple', -0.9816473130981943),
 ('my', -0.9711350992320826),
 ('u', -0.9425440398961344),
 ('he', -0.9103018053922919),
 ('i', -0.8998122273260664),
 ('ask', -0.8890145983450933),
 ("i'm", -0.8524625763251584),
 ('lose', -0.8517118308916841),
 ('said', -0.8500266948290822),
 ('ME', -0.839989670993406),
 ('good', -0.8379534186582519)]

In [18]:
# 逆で胡散臭いランキング
F[-20:]

[('claim', 1.4933937873007574),
 ('FREE', 1.5311685698234254),
 ('85233', 1.5597943544196624),
 ('FREE>Ringtone!Reply', 1.5597943544196624),
 ('-', 1.6076169970280638),
 ("let's", 1.6404697476557395),
 ('84484', 1.715317726901818),
 ('ringtoneking', 1.715317726901818),
 ('146tf150p', 1.734993477949411),
 ('2/2', 1.734993477949411),
 ('text', 1.75971235360272),
 ('service', 1.8899161812030403),
 ('&', 1.891587797841576),
 ('won', 1.9249391378528136),
 ('mobile', 1.9779817776242663),
 ('txt', 2.053966309086186),
 ('now!', 2.0779929639153956),
 ('STOP', 2.132429602315804),
 ('Call', 2.3701368026425973),
 ('Txt', 2.37855614106501)]